In [2]:
import sys

sys.path.insert(0, "..")
from src.settings import StyleSettings
from src.data.data_tools import StyleDataset
import numpy as np
# import seaborn as sns


# Load the data

In [3]:
settings = StyleSettings()
traindataset = StyleDataset([settings.trainpath])
testdataset = StyleDataset([settings.testpath])


We have 419 batches in the dataset

In [4]:
len(traindataset) // 32


419

In [5]:
x, y = traindataset[42]
x, y


('Lace is an openwork fabric , patterned with open holes in the work , made by machine or by hand.',
 'wiki')

Every batch is a `Tuple[str, str]` of a sentence and a label. We can see this is a classification task.
The task is, to classify sentences in four categories.
Lets build a vocabulary by copy-pasting the code we used before

In [6]:
from src.models import tokenizer

corpus = []
for i in range(len(traindataset)):
    x = tokenizer.clean(traindataset[i][0])
    corpus.append(x)
v = tokenizer.build_vocab(corpus, max=20000)
len(v)


2022-06-21 18:13:06.776 | INFO     | src.models.tokenizer:build_vocab:27 - Found 19306 tokens


19308

We need to cast the labels to an integers. You can use this dictionary to do so:

In [7]:
d = {"humor": 0, "reuters": 1, "wiki": 2, "proverbs": 3}
d[y]


2

# Question 1
Figure out, for every class, what accuracy you should expect if the model would guess blind on the train set

In [8]:
# TODO ~ about 4 lines of code
label_dict = {
    "humor": 0,
    "reuters": 0,
    "wiki": 0,
    "proverbs": 0
}

for row in traindataset.dataset:
    label_dict[row[1]] += 1

print(label_dict)

for k in label_dict:
    label_dict[k] =  label_dict[k] / len(traindataset.dataset) * 100

print(label_dict)


{'humor': 4213, 'reuters': 4186, 'wiki': 4181, 'proverbs': 831}
{'humor': 31.414510476474533, 'reuters': 31.21318320781448, 'wiki': 31.175900380284844, 'proverbs': 6.196405935426143}


Reflect on what you see. What does this mean? What implications does this have? Why is that good/bad?
Are there things down the line that could cause a problem?

### ANSWER
Due to the disbalance in the classes (4th to be precise) the model will probably tend to fit on the first three classes as these are easier to guess. Mostly the minority class will cause problems if we need to predict these. 

# Question 2 : Implement a preprocessor

We can inherit from `tokenizer.Preprocessor`
Only thing we need to adjust is the `cast_label` function.
 
- create a StylePreprocessor class
- inherit from Preprocessor
- create a new cast_label function for this dataset

In [9]:
# TODO ~ about 4 lines of code
class StylePreprocessor(tokenizer.Preprocessor):
    def cast_label(self, label: str) -> int:
        if label == "humor":
            return 0
        elif label == "reuters":
            return 1
        elif label == "wiki":
            return 2
        elif label == "proverbs":
            return 3

To test the preprocessor:

In [10]:
preprocessor = StylePreprocessor(max=100, vocab=v, clean=tokenizer.clean)
preprocessor([(x, y)])


(tensor([[4929,  854,   32,   15,  499,   21, 8496,  890]], dtype=torch.int32),
 tensor([2]))

# Set up the model
We can re-use the BaseDatastreamer

In [11]:
from src.data import data_tools

trainstreamer = data_tools.BaseDatastreamer(
    dataset=traindataset, batchsize=32, preprocessor=preprocessor
).stream()
teststreamer = data_tools.BaseDatastreamer(
    dataset=testdataset, batchsize=32, preprocessor=preprocessor
).stream()


In [12]:
x, y = next(trainstreamer)
x.shape, y


(torch.Size([32, 29]),
 tensor([1, 1, 0, 3, 2, 2, 2, 1, 2, 0, 2, 1, 1, 0, 2, 2, 0, 1, 0, 0, 0, 2, 0, 1,
         1, 1, 1, 3, 0, 1, 0, 0]))

## Question 3 : Metrics, loss
Select proper metrics and a loss function.

Bonus: implement an additional metric function that is relevant for this case.

In [26]:
from src.models import metrics
from sklearn.metrics import f1_score
import torch
# TODO ~ 2 lines of code

metric = metrics.F1Score()
loss_fn = torch.nn.CrossEntropyLoss()

Tensor = torch.Tensor

class WeightedF1Score(metrics.Metric):
    def __repr__(self) -> str:
        return "WeightedF1Score"

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        yhat = yhat.argmax(dim=1)
        score = f1_score(y, yhat, average="weighted")
        return torch.tensor(score)



In [27]:
type(metric)

src.models.metrics.F1Score

## Question 4 : Basemodel
Create a base model. It does not need to be naive; you could re-use the
NLP models we used for the IMDB.

I suggest to start with a hidden size of about 128.
Use a config dictionary, or a gin file, both are fine.

Bonus points if you create a Trax model in src.models, and even more if you add a trax training loop.

In [23]:
log_dir = settings.log_dir
# TODO between 2 and 8 lines of code, depending on your setup
# Assuming you load your model in one line of code from src.models.rnn
from src.models import rnn

config= {
    "vocab": len(v),
    "hidden_size": 128,
    "num_layers": 3,
    "dropout": 0.1,
    "output_size": 4,
}

model = rnn.NLPmodel(config)


Set up the trainloop.

- Give the lenght of the traindataset, how many batches of 32 can you get out of it?
- If you take a short amount of train_steps (eg 25) for every epoch, how many epochs do you need to cover the complete dataset?
- What amount of epochs do you need to run the loop with trainsteps=25 to cover the complete traindataset once? 
- answer the questions above, and pick a reasonable epoch lenght

Start with a default learning_rate of 1e-3 and an Adam optimizer.

In [65]:
from src.training import train_model

model = train_model.trainloop(
    epochs=50,
    model=model,
    metrics=[metric],
    optimizer=torch.optim.Adam,
    learning_rate=1e-3,
    loss_fn=loss_fn,
    train_dataloader=trainstreamer,
    test_dataloader=teststreamer,
    log_dir=log_dir,
    train_steps=25,
    eval_steps=25,
)


2022-06-21 19:31:28.404 | INFO     | src.data.data_tools:dir_add_timestamp:66 - Logging to ../tune/20220621-1931
100%|██████████| 25/25 [00:03<00:00,  6.59it/s]
2022-06-21 19:31:32.826 | INFO     | src.training.train_model:trainloop:164 - Epoch 0 train 1.2320 test 1.1595 metric ['0.2722']
100%|██████████| 25/25 [00:01<00:00, 13.00it/s]
2022-06-21 19:31:35.144 | INFO     | src.training.train_model:trainloop:164 - Epoch 1 train 1.0748 test 0.9884 metric ['0.3376']
100%|██████████| 25/25 [00:01<00:00, 15.88it/s]
2022-06-21 19:31:37.137 | INFO     | src.training.train_model:trainloop:164 - Epoch 2 train 0.9988 test 0.8866 metric ['0.4141']
100%|██████████| 25/25 [00:01<00:00, 16.88it/s]
2022-06-21 19:31:39.004 | INFO     | src.training.train_model:trainloop:164 - Epoch 3 train 0.8833 test 0.8896 metric ['0.3879']
100%|██████████| 25/25 [00:01<00:00, 18.03it/s]
2022-06-21 19:31:40.798 | INFO     | src.training.train_model:trainloop:164 - Epoch 4 train 0.8719 test 0.8504 metric ['0.5470']
10

Save an image from the training in tensorboard in the `figures` folder.
Explain what you are seeing.

In [70]:
import math
print(
    f'''
    1.  {len(traindataset)} datapoints divided by batches of size 32 equals {len(traindataset) / 32} batches before the dataset could be fully seen.
        Rounded up this would become 420
    2.  Depending on the way we look at the question it can either be {(len(traindataset)  + len(testdataset)) / (25 * 32)} or 21 epochs. 
        When we consider the train and test sets to be both run through the trainstep() function.
        If we consider the trainset in the evalbatches() and the trainset in the trainstep() functions
        then we take the max between those two for a total of {max(len(testdataset) / (25*32),
     len(traindataset) / (25*32))} (17 when rounded) epochs.
    3.  {len(traindataset) / (25*32)} or 17 rounded up.
    4.  25 epochs should be reasonable maybe 50 to be sure we have seen more combinations within the dataset. 
        If we want to go big and come close to the total amount of permutations the traindataset contains
        we can go up to the humongous number below given basic combinatoric math 
        (https://statisticsbyjim.com/probability/permutations-probabilities/): 
        {math.factorial(len(traindataset)) // math.factorial((len(traindataset) - (25*32)))}
    5.  in the image "tune/20220621-1931"  (could not save the image to the figure folder, sorry!)
        we can see only marginally smaal improvements after epoch 25.
        Even worse we see signals of overfitting where the test dataset loss is not improving (only wobbling)
        the train loss is decreasing comparatively a lot (the two are even diverging starting from epoch 45).
        This indicates that the model is learning the patterns in the trainset and not generalizing on the data.
    '''
)


    1.  13411 datapoints divided by batches of size 32 equals 419.09375 batches before the dataset could be fully seen.
        Rounded up this would become 420
    2.  Depending on the way we look at the question it can either be 20.955 or 21 epochs. 
        When we consider the train and test sets to be both run through the trainstep() function.
        If we consider the trainset in the evalbatches() and the trainset in the trainstep() functions
        then we take the max between those two for a total of 16.76375 (17 when rounded) epochs.
    3.  16.76375 or 17 rounded up.
    4.  25 epochs should be reasonable maybe 50 to be sure we have seen more combinations within the dataset. 
        If we want to go big and come close to the total amount of permutations the traindataset contains
        we can go up to the humongous number below given basic combinatoric math 
        (https://statisticsbyjim.com/probability/permutations-probabilities/): 
        25535954275326972078962366

# 5 Evaluate the basemodel
Create a confusion matrix with the code below

In [71]:
from sklearn.metrics import confusion_matrix

y_true = []
y_pred = []

for _ in range(10):
    X, y = next(teststreamer)
    yhat = model(X)
    yhat = yhat.argmax(dim=1)
    y_pred.append(yhat.tolist())
    y_true.append(y.tolist())

yhat = [x for y in y_pred for x in y]
y = [x for y in y_true for x in y]

cfm = confusion_matrix(y, yhat)
cfm_norm = cfm / np.sum(cfm, axis=1, keepdims=True)
plot = sns.heatmap(cfm_norm, annot=cfm_norm, fmt=".3f")
plot.set(xlabel="Predicted", ylabel="Target")


NameError: name 'sns' is not defined

Save this in the figures folder.
Interpret this. 

- What is going on?
- What is a good metric here?
- how is your answer to Q1 relevant here?
- Is there something you could do to fix/improve things, after you see these results?

# 6 Tune the model
Don't overdo this.
More is not better.

Bonus points for things like:
- Attention layers
- Trax architecture including a functioning training loop

Keep it small! It's better to present 2 or 3 sane experiments that are structured and thought trough, than 25 random guesses. You can test more, but select 2 or 3 of the best alternatives you researched, with a rationale why this works better.

Keep it concise; explain:
- what you changed
- why you thought that was a good idea  
- what the impact was (visualise or numeric)
- explain the impact

You dont need to get a perfect score; curiousity driven research that fails is fine.
The insight into what is happening is more important than the quantity.

Keep logs of your settings;
either use gin, or save configs, or both :)
Store images in the `figures` folder.